In [2]:
# %matplotlib widget

from __future__ import annotations

import re
from collections import defaultdict
from dataclasses import dataclass, field
from itertools import permutations, product
from math import inf
from random import choice

import matplotlib.colors as mcolors
import matplotlib.pyplot as plt
import networkx as nx
import numpy as np
import numpy.typing as npt
from mpl_toolkits.mplot3d import axes3d
from numpy import int_, object_
from numpy.typing import NDArray
from test_utilities import run_tests_params
from util import print_hex

COLORS = list(mcolors.CSS4_COLORS.keys())

<link href="style.css" rel="stylesheet"></link>
<article class="day-desc read-aloud"><h2>--- Day 11: Chronal Charge ---</h2><p>You watch the Elves and their sleigh fade into the distance as they head toward the North Pole.</p>
<p>Actually, you're the one fading. The <span title="wheeeeeeeeeeeeeeeeee">falling sensation</span> returns.</p>
<p>The low fuel warning light is illuminated on your wrist-mounted device. Tapping it once causes it to project a hologram of the situation: a <em>300x300</em> grid of fuel cells and their current power levels, some negative. You're not sure what negative power means in the context of time travel, but it can't be good.</p>
<p>Each fuel cell has a coordinate ranging <em>from 1 to 300</em> in both the X (horizontal) and Y (vertical) direction.  In <code>X,Y</code> notation, the top-left cell is <code>1,1</code>, and the top-right cell is <code>300,1</code>.</p>
<p>The interface lets you select <em>any 3x3 square</em> of fuel cells. To increase your chances of getting to your destination, you decide to choose the 3x3 square with the <em>largest total power</em>.</p>
<p>The power level in a given fuel cell can be found through the following process:</p>
<ul>
<li>Find the fuel cell's <em>rack ID</em>, which is its <em>X coordinate plus 10</em>.</li>
<li>Begin with a power level of the <em>rack ID</em> times the <em>Y coordinate</em>.</li>
<li>Increase the power level by the value of the <em>grid serial number</em> (your puzzle input).</li>
<li>Set the power level to itself multiplied by the <em>rack ID</em>.</li>
<li>Keep only the <em>hundreds digit</em> of the power level (so <code>12<em>3</em>45</code> becomes <code>3</code>; numbers with no hundreds digit become <code>0</code>).</li>
<li><em>Subtract 5</em> from the power level.</li>
</ul>
<p>For example, to find the power level of the fuel cell at <code>3,5</code> in a grid with serial number <code>8</code>:</p>
<ul>
<li>The rack ID is <code>3 + 10 = <em>13</em></code>.</li>
<li>The power level starts at <code>13 * 5 = <em>65</em></code>.</li>
<li>Adding the serial number produces <code>65 + 8 = <em>73</em></code>.</li>
<li>Multiplying by the rack ID produces <code>73 * 13 = <em>949</em></code>.</li>
<li>The hundreds digit of <code><em>9</em>49</code> is <code><em>9</em></code>.</li>
<li>Subtracting 5 produces <code>9 - 5 = <em>4</em></code>.</li>
</ul>
<p>So, the power level of this fuel cell is <code><em>4</em></code>.</p>
<p>Here are some more example power levels:</p>
<ul>
<li>Fuel cell at &nbsp;<code>122,79</code>, grid serial number <code>57</code>: power level <code>-5</code>.</li>
<li>Fuel cell at <code>217,196</code>, grid serial number <code>39</code>: power level &nbsp;<code>0</code>.</li>
<li>Fuel cell at <code>101,153</code>, grid serial number <code>71</code>: power level &nbsp;<code>4</code>.</li>
</ul>
<p>Your goal is to find the 3x3 square which has the largest total power. The square must be entirely within the 300x300 grid. Identify this square using the <code>X,Y</code> coordinate of its <em>top-left fuel cell</em>. For example:</p>
<p>For grid serial number <code>18</code>, the largest total 3x3 square has a top-left corner of <code><em>33,45</em></code> (with a total power of <code>29</code>); these fuel cells appear in the middle of this 5x5 region:</p>
<pre><code>-2  -4   4   4   4
-4  <em> 4   4   4  </em>-5
 4  <em> 3   3   4  </em>-4
 1  <em> 1   2   4  </em>-3
-1   0   2  -5  -2
</code></pre>
<p>For grid serial number <code>42</code>, the largest 3x3 square's top-left is <code><em>21,61</em></code> (with a total power of <code>30</code>); they are in the middle of this region:</p>
<pre><code>-3   4   2   2   2
-4  <em> 4   3   3  </em> 4
-5  <em> 3   3   4  </em>-4
 4  <em> 3   3   4  </em>-3
 3   3   3  -5  -1
</code></pre>
<p><em>What is the <code>X,Y</code> coordinate of the top-left fuel cell of the 3x3 square with the largest total power?</em></p>
</article>


In [3]:
tests_power_level_fuelcell = [
    {
        "name": "Example 1",
        "x": 3,
        "y": 5,
        "serialnumber": 8,
        "expected": 4,
    },
    {
        "name": "Example 2",
        "x": 122,
        "y": 79,
        "serialnumber": 57,
        "expected": -5,
    },
    {
        "name": "Example 3",
        "x": 217,
        "y": 196,
        "serialnumber": 39,
        "expected": 0,
    },
    {
        "name": "Example 4",
        "x": 101,
        "y": 153,
        "serialnumber": 71,
        "expected": 4,
    },
    {
        "name": "Example 5",
        "x": 33,
        "y": 45,
        "serialnumber": 18,
        "expected": 4,
    },
]


def power_level_fuelcell(x: int, y: int, serialnumber: int) -> int:
    # Find the fuel cell's rack ID, which is its X coordinate plus 10.
    rack_id = x + 10
    # Begin with a power level of the rack ID times the Y coordinate.
    power_level = rack_id * y
    # Increase the power level by the value of the grid serial number (your puzzle input).
    power_level += serialnumber
    # Set the power level to itself multiplied by the rack ID.
    power_level *= rack_id
    # Keep only the hundreds digit of the power level
    # (so 12345 becomes 3; numbers with no hundreds digit become 0).
    power_level = (power_level // 100) % 10
    # Subtract 5 from the power level.
    return power_level - 5


run_tests_params(power_level_fuelcell, tests_power_level_fuelcell)


Test Example 1 passed, for power_level_fuelcell.
Test Example 2 passed, for power_level_fuelcell.
Test Example 3 passed, for power_level_fuelcell.
Test Example 4 passed, for power_level_fuelcell.
Test Example 5 passed, for power_level_fuelcell.
Success


In [4]:
tests_create_grid = [
    {
        "name": "Example 1",
        "serialnumber": 18,
        "expected": "33,45",
    },
    {
        "name": "Example 2",
        "serialnumber": 42,
        "expected": "21,61",
    },
]


def display_3x3_subgrid(m: npt.ArrayLike, x: int, y: int) -> None:
    print(m[x - 1 : x + 4, y - 1 : y + 4].T)


def max_3x3_subgrid(serialnumber: int) -> tuple[int, int]:
    grid = np.array(
        [
            [power_level_fuelcell(x, y, serialnumber) for y in range(1, 301)]
            for x in range(1, 301)
        ]
    )
    mx, my = max(
        ((x, y) for x, y in product(range(300 - 3), repeat=2)),
        key=lambda t: grid[t[0] : t[0] + 3, t[1] : t[1] + 3].sum(),
    )

    display_3x3_subgrid(grid, mx, my)
    return f"{mx + 1},{my + 1}"


run_tests_params(max_3x3_subgrid, tests_create_grid)


[[-2 -4  4  4  4]
 [-4  4  4  4 -5]
 [ 4  3  3  4 -4]
 [ 1  1  2  4 -3]
 [-1  0  2 -5 -2]]
Test Example 1 passed, for max_3x3_subgrid.
[[-3  4  2  2  2]
 [-4  4  3  3  4]
 [-5  3  3  4 -4]
 [ 4  3  3  4 -3]
 [ 3  3  3 -5 -1]]
Test Example 2 passed, for max_3x3_subgrid.
Success


In [5]:
puzzle = 1788
print(f"Part I: {max_3x3_subgrid(puzzle)}")

[[-1  4 -2  4  0]
 [-5  4  4  4 -5]
 [ 0  4 -1  4  0]
 [-4  4  4  4 -5]
 [ 1 -5 -1  4  0]]
Part I: 235,35


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>235,35</code>.</p><p class="day-success">The first half of this puzzle is complete! It provides one gold star: *</p>
<article class="day-desc"><h2 id="part2">--- Part Two ---</h2><p>You discover a dial on the side of the device; it seems to let you select a square of <em>any size</em>, not just 3x3. Sizes from 1x1 to 300x300 are supported.</p>
<p>Realizing this, you now must find the <em>square of any size with the largest total power</em>. Identify this square by including its size as a third parameter after the top-left coordinate: a 9x9 square with a top-left corner of <code>3,5</code> is identified as <code>3,5,9</code>.</p>
<p>For example:</p>
<ul>
<li>For grid serial number <code>18</code>, the largest total square (with a total power of <code>113</code>) is 16x16 and has a top-left corner of <code>90,269</code>, so its identifier is <code><em>90,269,16</em></code>.</li>
<li>For grid serial number <code>42</code>, the largest total square (with a total power of <code>119</code>) is 12x12 and has a top-left corner of <code>232,251</code>, so its identifier is <code><em>232,251,12</em></code>.</li>
</ul>
<p><em>What is the <code>X,Y,size</code> identifier of the square with the largest total power?</em></p>
</article>

</main>


In [25]:
from tabulate import tabulate


tests_create_grid = [
    {
        "name": "Example 1",
        "serialnumber": 18,
        "expected": "90,269,16",
    },
    {
        "name": "Example 2",
        "serialnumber": 42,
        "expected": "232,251,12",
    },
]


def display_nxn_subgrid(m: npt.ArrayLike, n: int, x: int, y: int) -> None:
    print(m[x - 1 : x + n - 1, y - 1 : y + n - 1].T)


def max_nxn_subgrid_I(serialnumber: int) -> tuple[int, int]:
    grid = np.array(
        [
            [power_level_fuelcell(x, y, serialnumber) for y in range(1, 301)]
            for x in range(1, 301)
        ]
    )
    _, mx, my, n = max(
        (grid[x : x + n, y : y + n].sum(), x, y, n)
        for n in range(1, 301)
        for x, y in product(range(300 - n), repeat=2)
    )

    display_nxn_subgrid(grid, n, mx, my)
    return f"{mx + 1},{my + 1},{n}"


def max_nxn_subgrid(serialnumber: int) -> tuple[int, int]:
    grid = [
        [power_level_fuelcell(x, y, serialnumber) for y in range(1, 301)]
        for x in range(1, 301)
    ]

    summedarray = [[0] * 300 for _ in range(300)]
    for i in range(1, 300):
        summedarray[i][0] = grid[i - 1][0] + summedarray[i - 1][0]
        summedarray[0][i] += grid[0][i - 1] + summedarray[0][i - 1]

    for x, y in product(range(1, 300), repeat=2):
        summedarray[x][y] = (
            grid[x][y]
            + summedarray[x - 1][y]
            + summedarray[x][y - 1]
            - summedarray[x - 1][y - 1]
        )

    _, max_x, max_y, max_n = max(
        (
            summedarray[x][y]
            + summedarray[x + n][y + n]
            - summedarray[x + n][y]
            - summedarray[x][y + n],
            x,
            y,
            n,
        )
        for n in range(1, 301)
        for x, y in product(range(300 - n), repeat=2)
    )
    return f"{max_x + 2},{max_y + 2},{max_n}"


run_tests_params(max_nxn_subgrid, tests_create_grid)  # no spaces


Test Example 1 passed, for max_nxn_subgrid.
Test Example 2 passed, for max_nxn_subgrid.
Success


In [26]:
print(f"Part II: {max_nxn_subgrid(puzzle)}")

Part II: 142,265,7


<link href="style.css" rel="stylesheet"></link>
<main>

<p>Your puzzle answer was <code>142,265,7</code>.</p><p class="day-success">Both parts of this puzzle are complete! They provide two gold stars: **</p>

</main>
